# Most frequent builds

See what builds are most commonly invoked by developers, e.g. `clean assemble`, `test` or `check`. You can [set up the URL and a token for your Gradle Enterprise instance](https://github.com/gabrielfeo/gradle-enterprise-api-kotlin/blob/main/README.md#setup) and run this notebook as-is for your own project.

This is a simple example of something you can do with the API. It could bring insights, for example:

- "Our developers frequently `clean` together with `assemble`. We should ask them why, because they shouldn't have to. Just an old habit from Maven or are they working around a build issue we don't know about?"
- "Some are doing `check` builds locally, which we set up to trigger our notably slow legacy tests. We should suggest they run `test` instead, leaving `check` for CI to run."

This notebook will take you through using gradle-enterprise-api-kotlin in Jupyter, but it won't get into what a notebook is and how to run it. If you're not familiar with Jupyter:
- [Kotlin for data science overview](https://kotlinlang.org/docs/data-science-overview.html)
- [Kotlin for Jupyter notebooks](https://github.com/cheptsov/kotlin-jupyter-demo/blob/master/index.ipynb)

## Setup

`%use` is [line magic](https://github.com/Kotlin/kotlin-jupyter#line-magics) of the Kotlin kernel that can do much more than adding the library. To illustrate, this setup:

```kotlin
@file:DependsOn("com.github.gabrielfeo:gradle-enterprise-api-kotlin:0.15.1")

import com.gabrielfeo.gradle.enterprise.api.*
import com.gabrielfeo.gradle.enterprise.api.model.*
```

can be entirely replaced by:

```
%use gradle-enterprise-api-kotlin
```

In [1]:
%useLatestDescriptors
%use gradle-enterprise-api-kotlin(version=0.15.1)
%use coroutines(v=1.7.1)

## Parameters

Change these to process a longer or shorter time range (and test faster).

In [2]:
import java.time.*

val startDate = LocalDate.now().minusWeeks(1)

val buildFilter: (GradleAttributes) -> Boolean = { build ->
    "LOCAL" in build.tags
        && !build.hasFailed
}

In [3]:
// A utility to print progress as builds are fetched. You may ignore this.

fun <T> Flow<T>.printProgress(produceMsg: (i: Int, current: T) -> String): Flow<T> {
    var i = -1
    var current: T? = null
    fun printIt() {
        val msg = current?.let { produceMsg(i, it) } ?: "Waiting for elements..."
        print("\r$msg".padEnd(100))
    }
    val periodicPrinter = GlobalScope.launch {
        while (true) {
            printIt()
            delay(500)
        }
    }
    return onEach {
        i++
        current = it
    }.onCompletion {
        periodicPrinter.cancelAndJoin()
        printIt()
        println("\nEnd")
    }
}

## Fetch builds

[getGradleAttributesFlow][1] is a utility to call `/api/builds` and join each ID with `/api/builds/{id}/gradle-attributes` at once.

[1]: https://gabrielfeo.github.io/gradle-enterprise-api-kotlin/gradle-enterprise-api-kotlin/com.gabrielfeo.gradle.enterprise.api/get-gradle-attributes-flow.html

In [4]:
import java.time.temporal.*
import java.util.LinkedList

val builds: List<GradleAttributes> = runBlocking {
    val startMilli = startDate.atStartOfDay(ZoneId.of("UTC")).toInstant().toEpochMilli()
    GradleEnterprise.api.getGradleAttributesFlow(since = startMilli)
        .filter(buildFilter)
        .printProgress { i, build ->
            val buildDate = Instant.ofEpochMilli(build.buildStartTime).atOffset(ZoneOffset.UTC)
            String.format("Fetched %09d builds, currently %s", i + 1, buildDate)
        }.toList(LinkedList())
}

Fetched 000006412 builds, currently 2023-05-19T19:29:58.350Z                                                      
End


## Tables

We'll now use [Kotlin/dataframe](https://github.com/Kotlin/dataframe) to visualize data

In [5]:
%use dataframe(v=0.10.0)

Use `List.toDataFrame` to create a table of builds

In [6]:
val buildCounts = builds.toDataFrame {
    "tasks" from { build ->
        val tasks = build.requestedTasks.joinToString(" ").trim(':')
        if (tasks.isNotBlank()) tasks
        else "IDE sync"
    }
}.groupBy("tasks").aggregate {
    count() into "count"
}.sortByDesc("count")

// Jupyter will render the last cell line
buildCounts

... showing only top 20 of 682 rows DataFrame: rowsCount = 682, columnsCount = 2

## Plotting

We'll use [Kotlin/kandy](https://github.com/Kotlin/kandy) for plotting the table. We'll only plot the top 5.

In [7]:
%use kandy(v=0.4.1)

In [8]:
plot(buildCounts.take(5)) {
    barsH { 
        x("count")
        y("tasks")
    }
    layout.size = 1000 to 250
}